## Бэггинг и случайный лес

Загрузите датасет digits с помощью функции load_digits из sklearn.datasets и подготовьте матрицу признаков X и ответы на обучающей выборке y (вам потребуются поля data и target в объекте, который возвращает load_digits).

Для оценки качества далее нужно будет использовать cross_val_score из sklearn.cross_validation с параметром cv=10. Функция cross_val_score будет возвращать numpy.ndarray, в котором будет k чисел - качество в каждом из k экспериментов k-fold cross validation. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод .mean() у массива, который возвращает cross_val_score.

In [62]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score 

In [2]:
digits = load_digits()

In [3]:
print(digits.keys()) 

dict_keys(['data', 'target', 'target_names', 'images', 'DESCR'])


In [4]:
x = digits['data'] 
y = digits['target'] 
print(x.shape, y.shape) 

(1797, 64) (1797,)


## Задание

__1.__ Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score.

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(x, y, random_state = 17, test_size = 0.3)

In [16]:
tree = DecisionTreeClassifier()

tree.fit(X_train, y_train)

print('Score: ', tree_fit.score(X_train, y_train))

Score:  1.0


In [17]:
print('mean: ', (cross_val_score(DecisionTreeClassifier(), X_train, y_train, scoring = 'r2', cv = 10)).mean());

mean:  0.642857351407


__2.__ Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100.

Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [10]:
from sklearn.ensemble import BaggingClassifier

In [11]:
clf = BaggingClassifier(n_estimators=100)
clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [18]:
print('mean: ', (cross_val_score(BaggingClassifier(n_estimators=100), X_train, y_train, scoring = 'r2', cv = 10)).mean());

mean:  0.873029329074


__3.__ Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех $d$ признаках, а на $\sqrt{d}$ случайных признаков.

Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [33]:
import math
max_features = math.sqrt(x.shape[1])
max_features

8.0

In [34]:
est = BaggingClassifier(n_estimators = 100, max_features = 8)
est.fit(X_train, y_train)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=8, max_samples=1.0,
         n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [35]:
print('mean: ', (cross_val_score(est, X_train, y_train, scoring = 'r2', cv = 10)).mean());

mean:  0.906015520181


__4.__ Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Попробуйте выбирать опять же $\sqrt{d}$ признаков. 

In [19]:
estimator = BaggingClassifier(DecisionTreeClassifier(max_features = 8), n_estimators = 100)
estimator.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=8, max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=100, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [20]:
print('mean: ', (cross_val_score(estimator, X_train, y_train, scoring = 'r2', cv = 10)).mean());

mean:  0.963767272083


__5.__ Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности постройте графики зависимости качества от значений параметров.

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [75]:
rf = RandomForestClassifier(n_estimators=100, max_features=8)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [76]:
print('mean: ', (cross_val_score(rf, X_train, y_train, scoring = 'r2', cv = 10)).mean());

mean:  0.950563385137


In [106]:
max_depth_values = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
max_features_values = [4, 5, 6, 7, 8, 10, 15, 30, 50, 55, 60]
forest_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

In [107]:
rf = RandomForestClassifier(n_estimators=100) 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
random_grid = GridSearchCV(rf, forest_params, cv=10, scoring = 'r2')

In [108]:
random_grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'max_features': [4, 5, 6, 7, 8, 10, 15, 30, 50, 55, 60]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [109]:
random_grid.best_params_, random_grid.best_score_

({'max_depth': 15, 'max_features': 8}, 0.96518031361250933)

In [114]:
rf = RandomForestClassifier(n_estimators=100, max_depth=15, max_features=8)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features=8, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [115]:
print('mean: ', (cross_val_score(rf, X_train, y_train, scoring = 'r2', cv = 10)).mean());

mean:  0.953554634611


Выберите верные утверждения:
1. **<font color='green'>Случайный лес сильно переобучается с ростом количества деревьев.</font>**
2. **<font color='green'>При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев.</font>**
3. **<font color='green'>С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.</font>**
4. **<font color='green'>При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.</font>**
5. При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.
6. При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.
7. **<font color='green'> При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).</font>**